# NISQ Experiment

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from qiskit import Aer
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.providers.aer import AerSimulator
from copy import deepcopy

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from utils import *
from experiments import *
from qiskit.providers.fake_provider import FakeCasablanca
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import random_unitary, Operator

In [3]:
#qk.IBMQ.save_account("a9ec3c697bede976217e9ff6c5c0f19a2511ea34a225bbe10f0c74bda39de845d16fa3f7c30b47c5714e2b695aded3cfd474e7c1b33056014fab89301b83d724", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend("ibmq_manila")

#backend = AerSimulator()

In [4]:
def execute_and_collect(circuit_list, backend, file_name):
    N = len(circuit_list)
    num_batches = N//500
    circuit_batch_list = [circuit_list[500*i: 500*(i+1)] for i in range(num_batches)]
    counts_list = []
    for i, circuit_batch in enumerate(tqdm(circuit_batch_list)):
        circuit_parcel_list = [circuit_batch[100*j: 100*(j+1)] for j in range(5)]
        job_list = []
        
        for circuit_parcel in circuit_parcel_list:
            job = qk.execute(circuit_parcel, backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
            job_list.append(job)
        
        result_list = []
        for job in tqdm(job_list):
            result_list.append(job.result())
            
        for result, circuit_parcel in zip(result_list, circuit_parcel_list):
            counts_list.extend([result.get_counts(circuit) for circuit in circuit_parcel])
        
        pickle.dump(counts_list, open(f"../../data/{file_name}_{i}.p", "wb"))   
        
    return counts_list  

## Pauli String Expectation Values

## Four Qubit QFT

In [42]:
def qft(n):
    circuit = qk.QuantumCircuit(n)
    for i in range(n):
        circuit.h(i)
        for j in range(i+1, n):
            circuit.cp(2*np.pi/2**(j-i+1), j, i)
            
    return circuit

In [40]:
n = 4
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = qft(n).reverse_bits()

N = 5000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)

In [41]:
counts_list = execute_and_collect(circuit_list, backend, 'fourQubits_QFT')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/fourQubits_QFT.p", "wb"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### More data

In [43]:
n = 4
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = qft(n).reverse_bits()

N = 5000-6**n + 2000
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

input_map_list = [input_map_list[0][-2000:], input_map_list[1][-2000:]]
circuit_list = circuit_list[-2000:]

In [45]:
counts_list = execute_and_collect(circuit_list, backend, 'fourQubits_QFT_more')
data = [input_map_list, counts_list]
pickle.dump(data, open("../../data/fourQubits_QFT_more.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Mock Data

### Two Qubits

In [ ]:
n = 2
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()
print(circuit_target)

N = 324
input_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        circuit = circuit.compose(circuit_target)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)
    
for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n)) 

In [ ]:
backend = FakeCasablanca()

job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()
counts_list = [result1.get_counts(circuit) for circuit in circuit_list[:100]]
counts_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:400]])

data = [input_list, counts_list[:N], counts_list[N:N+20], counts_list[N+20:N+20+d**2]]

pickle.dump(data, open("../../data/twoQubits_pauliStrings_corrMat_mock.p", "wb"))

### Three Qubits

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()
print(circuit_target)

N = 1000-20-d**2
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)


for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        circuit = circuit.compose(circuit_target)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)
    
for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n)) 

In [ ]:
job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job5 = qk.execute(circuit_list[400:500], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()
result5 = job5.result()

counts_list = [result1.get_counts(circuit) for circuit in circuit_list[:100]]
counts_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:400]])
counts_list.extend([result5.get_counts(circuit) for circuit in circuit_list[400:500]])

In [ ]:
job6 = qk.execute(circuit_list[500:600], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job7 = qk.execute(circuit_list[600:700], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job8 = qk.execute(circuit_list[700:800], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job9 = qk.execute(circuit_list[800:900], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job10 = qk.execute(circuit_list[900:1000], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result6 = job6.result()
result7 = job7.result()
result8 = job8.result()
result9 = job9.result()
result10 = job10.result()

counts_list.extend([result6.get_counts(circuit) for circuit in circuit_list[500:600]])
counts_list.extend([result7.get_counts(circuit) for circuit in circuit_list[600:700]])
counts_list.extend([result8.get_counts(circuit) for circuit in circuit_list[700:800]])
counts_list.extend([result9.get_counts(circuit) for circuit in circuit_list[800:900]])
counts_list.extend([result10.get_counts(circuit) for circuit in circuit_list[900:1000]])

data = [input_map_list, counts_list[:N], counts_list[N:N+20], counts_list[N+20:N+20+d**2]]

pickle.dump(data, open("../../data/threeQubits_pauliStrings_corrMat_mock.p", "wb"))

## Variational Circuit

## One Layer

In [35]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 1).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)

In [36]:
print(circuit_target)

      ┌────────────┐ ┌─────────────┐     ┌───┐
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├─────┤ X ├
      ├────────────┤ ├─────────────┤┌───┐└─┬─┘
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├──■──
     ┌┴────────────┴┐├─────────────┤└─┬─┘     
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■───────
     └──────────────┘└─────────────┘          


In [33]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Two Layers

In [38]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 2).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)

In [39]:
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐     ┌───┐
«q_0: ─┤ Rz(2.9525) ├─────┤ X ├
«     ┌┴────────────┤┌───┐└─┬─┘
«q_1: ┤ Rz(-3.0123) ├┤ X ├──■──
«     └─────────────┘└─┬─┘     
«q_2: ─────────────────■───────
«                              


In [40]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_2')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_2.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Three Layers

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 3).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3.p", "wb"))

## Four Layers

In [43]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 4).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [44]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_4')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_4.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Six Layers

In [6]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 6).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [7]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_6')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_6.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Nine Layers

In [8]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [13]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_12')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_12.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Twelve Layers

In [14]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 12, 0, 12).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [15]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_12')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_12.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## L2

In [7]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 6, 3, 6).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

     ┌─────────────┐┌──────────────┐          ┌───┐       ┌────────────┐ »
q_0: ┤ Ry(0.70279) ├┤ Rz(-0.83967) ├──────────┤ X ├───────┤ Ry(-1.887) ├─»
     ├─────────────┤└┬────────────┬┘┌───┐     └─┬─┘       ├────────────┤ »
q_1: ┤ Ry(-1.3117) ├─┤ Rz(-1.306) ├─┤ X ├───────■─────────┤ Ry(1.7918) ├─»
     ├─────────────┤┌┴────────────┤ └─┬─┘┌──────────────┐┌┴────────────┴┐»
q_2: ┤ Ry(-0.4276) ├┤ Rz(-2.2651) ├───■──┤ Ry(-0.27602) ├┤ Rz(0.089438) ├»
     └─────────────┘└─────────────┘      └──────────────┘└──────────────┘»
«     ┌─────────────┐          ┌───┐     ┌─────────────┐┌────────────┐     »
«q_0: ┤ Rz(-2.8497) ├──────────┤ X ├─────┤ Ry(-2.7329) ├┤ Rz(1.9377) ├─────»
«     ├─────────────┤┌───┐     └─┬─┘     ├─────────────┤├────────────┤┌───┐»
«q_1: ┤ Rz(0.58066) ├┤ X ├───────■───────┤ Ry(-2.0702) ├┤ Rz(2.9257) ├┤ X ├»
«     └─────────────┘└─┬─┘┌─────────────┐└┬────────────┤└────────────┘└─┬─┘»
«q_2: ─────────────────■──┤ Ry(0.67572) ├─┤ Rz(2.8204) ├────────────────■──»
«            

In [8]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3_2')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3_2.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## L3

In [11]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 9, 6, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐┌───────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.1576) ├┤ Rz(-0.030304) ├──────────┤ X ├─────┤ Ry(-1.5156) ├»
     ┌┴────────────┤└┬─────────────┬┘┌───┐     └─┬─┘     └┬────────────┤»
q_1: ┤ Ry(-2.5279) ├─┤ Rz(-2.3748) ├─┤ X ├───────■────────┤ Ry(2.5718) ├»
     ├─────────────┤ ├─────────────┴┐└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-1.2276) ├─┤ Rz(-0.37603) ├──■──┤ Ry(-2.9255) ├─┤ Rz(1.0212) ├»
     └─────────────┘ └──────────────┘     └─────────────┘ └────────────┘»
«     ┌─────────────┐          ┌───┐      ┌────────────┐┌────────────┐     »
«q_0: ┤ Rz(0.12609) ├──────────┤ X ├──────┤ Ry(2.9505) ├┤ Rz(2.4808) ├─────»
«     ├─────────────┤┌───┐     └─┬─┘     ┌┴────────────┤├────────────┤┌───┐»
«q_1: ┤ Rz(-1.1831) ├┤ X ├───────■───────┤ Ry(-1.9801) ├┤ Rz(2.7615) ├┤ X ├»
«     └─────────────┘└─┬─┘┌─────────────┐└┬────────────┤└────────────┘└─┬─┘»
«q_2: ─────────────────■──┤ Ry(0.29349) ├─┤ Rz(1.7287) ├────────────────■──»
«                   

In [12]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3_3')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3_3.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## L4

In [16]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 12, 9, 12).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

     ┌─────────────┐┌─────────────┐          ┌───┐       ┌────────────┐ »
q_0: ┤ Ry(-2.5856) ├┤ Rz(-1.0975) ├──────────┤ X ├───────┤ Ry(2.0655) ├─»
     └┬────────────┤├─────────────┤┌───┐     └─┬─┘      ┌┴────────────┤ »
q_1: ─┤ Ry(2.6507) ├┤ Rz(-2.8574) ├┤ X ├───────■────────┤ Ry(-1.4367) ├─»
     ┌┴────────────┤├─────────────┤└─┬─┘┌──────────────┐├─────────────┴┐»
q_2: ┤ Ry(0.61512) ├┤ Rz(-1.9102) ├──■──┤ Ry(-0.69946) ├┤ Rz(-0.90005) ├»
     └─────────────┘└─────────────┘     └──────────────┘└──────────────┘»
«     ┌─────────────┐          ┌───┐     ┌─────────────┐┌────────────┐     »
«q_0: ┤ Rz(0.26827) ├──────────┤ X ├─────┤ Ry(-2.6732) ├┤ Rz(-1.893) ├─────»
«     ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤├────────────┤┌───┐»
«q_1: ┤ Rz(-1.3764) ├┤ X ├───────■────────┤ Ry(1.8988) ├┤ Rz(1.7106) ├┤ X ├»
«     └─────────────┘└─┬─┘┌─────────────┐ ├────────────┤└────────────┘└─┬─┘»
«q_2: ─────────────────■──┤ Ry(-2.2561) ├─┤ Rz(3.0592) ├────────────────■──»
«                   

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3_4')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3_4.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]